### Take surface EN4 data, regrid to target grid, and save to zarr store

In [1]:
import xarray as xr
import xesmf as xe
from dask.diagnostics import ProgressBar

In [4]:
target = 'merra2'
rootdir_en4 = '/scratch/gpfs/graemem/en4/'
if target=='jra55':
    rootdir = '/projects/SOCCOM/'
elif target=='era5':
    rootdir = '/'
elif target=='merra2':
    rootdir='/'
rootdir_target=rootdir+'scratch/gpfs/graemem/'+target+'/'

In [5]:
grid_path = {'jra55':'areacella_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-5-0_gr.nc',
            'era5':'regular_grid.nc',
            'merra2':'regular_grid.nc'}
grid_target = xr.open_dataset(rootdir_target+grid_path[target])

In [6]:
if target == 'era5':
    grid_target = grid_target.rename({'longitude':'lon','latitude':'lat'})

In [7]:
grid_target

<xarray.Dataset>
Dimensions:    (lon_right: 576, lat: 361, lon: 576, lat_right: 361)
Coordinates:
  * lon        (lon) float64 -180.0 -179.4 -178.8 -178.1 ... 178.1 178.8 179.4
  * lat        (lat) float64 -90.0 -89.5 -89.0 -88.5 ... 88.5 89.0 89.5 90.0
  * lon_right  (lon_right) float64 -179.7 -179.1 -178.4 ... 178.4 179.1 179.7
  * lat_right  (lat_right) float64 -89.75 -89.25 -88.75 ... 89.25 89.75 90.25
Data variables:
    dxG        (lon_right, lat) float64 ...
    dyG        (lon, lat_right) float64 ...
    dxC        (lon, lat) float64 ...
    dyC        (lon, lat) float64 ...
    rC         (lon, lat) float64 ...
Attributes: (12/30)
    History:                           Original file generated: Fri May  8 00...
    Filename:                          MERRA2_100.tavgM_2d_ocn_Nx.198001.nc4
    Comment:                           GMAO filename: d5124_m2_jan79.tavg1_2d...
    Conventions:                       CF-1
    Institution:                       NASA Global Modeling and Assimilation ...
    References:                        http://gmao.gsfc.nasa.gov
    ...                                ...
    DataResolution:                    0.5 x 0.625
    Source:                            CVS tag: GEOSadas-5_12_4
    Contact:                           http://gmao.gsfc.nasa.gov
    identifier_product_doi:            10.5067/4IASLIDL8EEC
    RangeBeginningTime:                00:00:00.000000
    RangeEndingTime:                   23:59:59.000000

In [8]:
def preprocess(ds):
    return ds.isel(depth=0)
ds_en4 = xr.open_mfdataset(rootdir_en4+'EN.4.2.2/*.nc',preprocess=preprocess).squeeze()

In [9]:
ds_en4 = ds_en4.chunk({'time':120})
ds_en4

<xarray.Dataset>
Dimensions:                          (lat: 173, lon: 360, time: 1469, bnds: 2)
Coordinates:
    depth                            float32 5.022
  * lat                              (lat) float32 -83.0 -82.0 ... 88.0 89.0
  * lon                              (lon) float32 1.0 2.0 3.0 ... 359.0 360.0
  * time                             (time) datetime64[ns] 1900-01-16T12:00:0...
Dimensions without coordinates: bnds
Data variables:
    temperature                      (time, lat, lon) float32 dask.array<chunksize=(120, 173, 360), meta=np.ndarray>
    salinity                         (time, lat, lon) float32 dask.array<chunksize=(120, 173, 360), meta=np.ndarray>
    temperature_uncertainty          (time, lat, lon) float32 dask.array<chunksize=(120, 173, 360), meta=np.ndarray>
    salinity_uncertainty             (time, lat, lon) float32 dask.array<chunksize=(120, 173, 360), meta=np.ndarray>
    temperature_observation_weights  (time, lat, lon) float32 dask.array<chunksize=(120, 173, 360), meta=np.ndarray>
    salinity_observation_weights     (time, lat, lon) float32 dask.array<chunksize=(120, 173, 360), meta=np.ndarray>
    time_bnds                        (time, bnds) datetime64[ns] dask.array<chunksize=(120, 2), meta=np.ndarray>
    depth_bnds                       (time, bnds) float32 dask.array<chunksize=(120, 2), meta=np.ndarray>
Attributes: (12/22)
    Conventions:            CF-1.0
    title:                  Temperature and salinity analysis
    DSD_entry_id:           UKMO-L4UHFnd-GLOB-v01
    references:             Website and paper: https://www.metoffice.gov.uk/h...
    institution:            UK Met Office
    contact:                Rachel Killick - rachel.killick@metoffice.gov.uk
    ...                     ...
    southernmost_latitude:  -90.5
    northernmost_latitude:  89.5
    westernmost_longitude:  0.5
    easternmost_longitude:  362.5
    file_quality_index:     0
    licence:                EN4 is distributed under the Non Commercial Gover...

In [10]:
regridder = xe.Regridder(ds_en4,grid_target,'bilinear',periodic=True)
ds_regridded = regridder(ds_en4)

In [11]:
with ProgressBar():
    ds_regridded.to_zarr(rootdir_en4+'EN.4.2.2_regridded/'+target+'.zarr',mode='w')

[########################################] | 100% Completed | 14min 14.7s


## Check

In [7]:
ds = xr.open_zarr(rootdir_en4+'EN.4.2.2_regridded/'+target+'.zarr')

In [9]:
ds['salinity'].isel(time=0).plot()

AttributeError: 'NoneType' object has no attribute 'matplotlib'